# Head Pose Estimation

## Explanation of Approach

The overall approach to the problem can broadly be split into three main sections. First, there is the detection of the head/face in the images that are supplied. Secondly, we are extract 2-dimensional landmarks on the face at a number of keypoints. Once we have these keypoints, we use a 3-dimensional approximation of the face to estimate the pose of the head in 3-d described by a transition $t$ and a rotoational matrix $R$. This pose and 3-dimensional line from a suitable keypoint can then be used to describe the pose in terms of the 4 or 8 directions. Each of these stages will be descibed in detail in the sections below.

### Face Detection

Dlib's landmark detection is based on the output of a face detector that returns the position of the four courners of the face. To ensure reliable performance of the landmark detection, a stable face detector is used. In this particular implementation, I used the HOG face detector available with dlib because of its sufficient accuracy and output compatibility with the landmark detction model that I would use.

### Landmark Detection

I used the shape_predictor_68_face_landmarks model available at dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 that is pre-trained on the ibug 300-W dataset as described here: https://github.com/davisking/dlib-models. This gives 68 facial landmarks, only 6 are required for the face estimation and these were the following: nose, chin, left corner of left eye, right corner of right eye, left corner of mouth and right corener of mouth. These are the facial landmarks that are available in the 3-d world model (a generic model was found online) of the face that is an approximate model of a human face in an arbitrary reference frame and is sufficient for this particular application.


### Pose Estimation

Once we have the 2-d landmarks in the camera reference frame and the 3-d point approximation of the face. We are trying to find the rotational matrix and translational vector that govern the projection of a point $[W_1, W_2, W_3]$ in the world frame onto the camera frame. From knowledge of computer vision and projection, this can be approximated to within a scale factor $s$ by use of an appropriate solver. In this case I chose to use the cv2.solvePnPRansac that uses RANSAC to provide a more robust estimate of pose by solving the equation below for the rotational matrix $R$ and translation vector $t$:

$$
\begin{pmatrix} x \\ y \\ 1\end{pmatrix} = s\begin{pmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{pmatrix}\left( \begin{array}{ccc|c} r_{00} & r_{01} & r_{02} & t_0 \\ r_{10} & r_{11} & r_{12} & t_1\\ r_{20} & r_{21} & r_{22} & t_2 \end{array} \right)\begin{pmatrix} W_1 \\ W_2 \\W_3\\ 1\end{pmatrix}
$$

Given the point in the world co-ordinate system and the corresponding 2-d point projected in the camera frame we find $R$
and $t$. Then we use an arbitrary line pointing out of the nose in the world co-ordinate frame $[nose_{Wx},nose_{Wy},nose_{Wz}+500]$ and by projecting this back onto the camera co-ordinate system we can draw a pointer that indicates the direction that one is facing. When we reproject this arbitrary vector we have a point in 2-d camera frame: $[noseend2d_x,noseend2d_y]$. Then we can get the horizonatal (x) and vertical (y) difference between the start and end point. Using this and the arctan2 function, we get a quadrant specific angle that we can then map to one of the directions, in this case: up, down, left, right, left up, right up, left down and right down.

$$ \theta = arctan((nosestart2d_y-noseend2d_y)/(nosestart2d_x-noseend2d_x))$$
## Dependencies

The code was run using the following libraries:

#### OpenCV v3
Used for various image processing and computer vision tools
#### Numpy

#### dlib
Used for the face detection and landmark prediction
#### dload
Used to download the models used for the landmark prediction
#### bz2
Used to extract the downloaded the models

## Source Code

### Downloading and importing of libraries

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 20 16:48:41 2020

@author: Nash
"""

# We Import the necessary packages needed 
import cv2 
import numpy as np 
import dlib 
import dload
import bz2
#download pre-trained landmarks model 

#dload.save('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2')
filepath = 'shape_predictor_68_face_landmarks.dat.bz2'
zipfile = bz2.BZ2File(filepath) 
data = zipfile.read() 
newfilepath = filepath[:-4] 
open(newfilepath, 'wb').write(data) 

99693937

### Pose detection on video stream

In [4]:
#capture webcam images  
cap = cv2.VideoCapture(0) 
# We initialise detector of dlib 

#HOG face detector
detector = dlib.get_frontal_face_detector() 

#Pretrained landmark predictor
predictor = dlib.shape_predictor(newfilepath) 

#known approximation for 3d world model of face keypoints
points_3d = np.array([[0.0,0.0,0.0],#nose
             [0.0,-330.0,-65.0],#chin
             [-225.0, 170.0, -135.0],#left corner of left eye
             [225.0, 170.0, -135.0],#right corner of right eye
             [-150.0, -150.0, -125.0],#left corner of mouth
             [150.0, -150.0, -125.0]]) #right corener of mouth
#assume no distortion to simplify ccamera matrix
distortion = np.zeros((4,1))
# Start receiving webcam feed frame by frame 
while True: 
    _, frame = cap.read() 
   # Convert to grayscale conversion 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    #run face detector on frame
    faces = detector(gray)
    
    #approximate the camera matrix parameters for projections
    sz = gray.shape
    focal_l = sz[1] #use width as proxy for fx and fy
    c = (sz[1]/2, sz[0]/2) #center of image
    camera_params = np.array(
                         [[focal_l, 0, c[0]],
                         [0, focal_l, c[1]],
                         [0, 0, 1]])

    for face in faces: 
        #get the face detection bounding box
        points_2d = []
        l = face.left() 
        t = face.top() 
        r = face.right() 
        b = face.bottom() 
        cv2.rectangle(frame, (l, t), (r, b), (255, 255, 0), 3) 
        landmarks = predictor(gray, face) 
       # Get the landmarks that we need 5/6 choose the landmarks corresponding to keypoints
       #from estimates of 3d model of a face in arbitrary reference frame
        for land in [30,8,36,45,48,54]: 
            x = landmarks.part(land).x 
            y = landmarks.part(land).y 
            
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            points_2d.append([x,y])
        points_2d = np.array(points_2d,dtype = "double")
        
        #get the rotational and translation matrices 
        (_,rotation, translation,_) =cv2.solvePnPRansac(points_3d, points_2d, camera_params, distortion, flags=cv2.SOLVEPNP_ITERATIVE)


        
        #what is the angle that the we are facing?
        #get the angle using a line from the nose pointing out projected to the 2d frame
        (nose_end_point2d, _) = cv2.projectPoints(np.array([(0.0, 0.0, 700.0)]), rotation, translation, camera_params, distortion)

            
        p1 = ( int(points_2d[0][0]), int(points_2d[0][1]))
        p2 = ( int(nose_end_point2d[0][0][0]), int(nose_end_point2d[0][0][1]))

        cv2.line(frame, p1, p2, (0,0,), 2)
       
        horizontal = p1[0]-p2[0]
        vertical = p1[1]-p2[1]
        angle = np.arctan2(vertical,horizontal)* 180 / np.pi
        
        #check what sector the angle is in
        
        if  angle >= -22.5 and angle <22.5:
            text = "left"
        elif angle >=22.5 and angle < 67.5:
            text = "left up"
        elif angle >=67.5 and angle < 112.5:
            text = "up"
        elif angle >=112.5 and angle < 157.5:
            text = "up right"
        elif angle >=157.5 or angle < -157.5:
            text = "right"
        elif angle >=-157.5 and angle < -112.5:
            text = "right down"
        elif angle >=-112.5 and angle < -67.5:
            text = "down"
        elif angle >=-67.5 and angle < -22.5:
            text = "left down"

            
        #display the angle and direction on the frame
        font = cv2.FONT_HERSHEY_PLAIN 
        cv2.putText(frame, str(int(angle)), tuple(p1),font, 1, (128, 255, 255), 2)
        cv2.putText(frame, text, tuple(p2),font, 1, (128, 255, 255), 2)
        
    cv2.imshow("Frame", frame) 
    key = cv2.waitKey(1) 
    
    if key == 27: 

        break # press esc the frame is destroyed 
        
cv2.destroyAllWindows()
cap.release()       

## Demo Video

Find the demo video at: https://drive.google.com/drive/folders/15dFgoeSjFGhnmyLcbawe_raJX0SpdvDC?usp=sharing

The number displayed on the nose tip is the angle $\theta$ described above.